In [7]:
import pandas as pd
from pandas import get_dummies
import numpy as np

In [8]:
train = pd.read_csv('train.tsv', sep='\t')
test = pd.read_csv('test.tsv', sep='\t')

In [3]:
train.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete with certificate of authenticity


In [4]:
train.shape

(1482535, 8)

In [3]:
labels = np.log1p(np.array(train['price'])+1)

## trying gensim word2vec

In [4]:
import stop_words
import re
import string

## tfidf

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
SIZE = 50

In [27]:
tv = TfidfVectorizer(max_features=SIZE, ngram_range=(1,5))

In [28]:
tv.fit(train.sample(n=100000)['item_description'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=50, min_df=1,
        ngram_range=(1, 5), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [29]:
train_sample = train.sample(n=10000)

In [30]:
desc_x = tv.transform(train_sample['item_description'])

In [31]:
y = np.log1p(np.array(train_sample['price'])+1)

## gensim w2v

In [24]:
train_sample.iloc[0,7].split(',')

['great condition',
 ' used about twice & swatched',
 ' smoke free home',
 ' authentic']

In [3]:
import gensim

In [5]:
w2v = gensim.models.Word2Vec(size = SIZE)

In [12]:
w2v.build_vocab(train_sample)

In [15]:
kw = w2v.wv

In [19]:
kw['great condition']

KeyError: "word 'great condition' not in vocabulary"

## model for description

In [32]:
from sklearn.model_selection import train_test_split

In [33]:
import xgboost as xgb

/home/lyan/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [34]:
train_x, test_x, train_y, test_y = train_test_split(desc_x, y)

In [35]:
dtrain = xgb.DMatrix(train_x, train_y)
dtest = xgb.DMatrix(test_x, test_y)

In [36]:
params = {}
params['objective'] = 'reg:linear'
params['nthread'] = 3
params['metric'] = 'rmse'
params['max_depth'] = 5
params['n_estimators'] = 500
params['silent'] = 1

watchlist = [(dtrain, 'train'), (dtest, 'test')]

In [37]:
bst = xgb.train(params, dtrain, 1000, watchlist, verbose_eval=100)

[0]	train-rmse:1.89864	test-rmse:1.90874
[100]	train-rmse:0.53204	test-rmse:0.724836
[200]	train-rmse:0.471284	test-rmse:0.738098
[300]	train-rmse:0.430165	test-rmse:0.749502
[400]	train-rmse:0.405911	test-rmse:0.756159
[500]	train-rmse:0.388239	test-rmse:0.759859
[600]	train-rmse:0.378161	test-rmse:0.763786
[700]	train-rmse:0.370868	test-rmse:0.766556
[800]	train-rmse:0.366202	test-rmse:0.769556
[900]	train-rmse:0.363124	test-rmse:0.771511


## making prediction

In [40]:
BATCH_SIZE = 100000

In [46]:
submission: pd.DataFrame = test[['test_id']]

In [38]:
test_x = tv.transform(test['item_description'])

,test_id,name,item_condition_id,category_name,brand_name,shipping,item_description
0,0,"Breast cancer ""I fight like a girl"" ring",1,Women/Jewelry/Rings,NaN,1,Size 7
1,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers",1,Other/Office supplies/Shipping Supplies,NaN,1,"25 pcs NEW 7.5""x12"" Kraft Bubble Mailers Lined..."
2,2,Coach bag,1,Vintage & Collectibles/Bags and Purses/Handbag,Coach,1,Brand new coach bag. Bought for [rm] at a Coac...
3,3,Floral Kimono,2,Women/Sweaters/Cardigan,NaN,0,-floral kimono -never worn -lightweight and pe...
4,4,Life after Death,3,Other/Books/Religion & Spirituality,NaN,1,Rediscovering life after the loss of a loved o...
